# Torrevieja Data Integration

In [ ]:
import os
import json
import pandas as pd

## Weather Station

Data about temperature (degree), humidity (%) and pressure (hPa) from weather station.

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "temperatura": {
            "sensor-id": [
                [ "unix-timestamp1", "value1" ],
                [ "unix-timestamp2", "value2" ]
            ],
        },
        "humedad": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        },
        "presion": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        }
    }
}
```

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `temperature`
- `humidity`
- `pressure`

In [ ]:
folder_path = "json_data/weather-station"
sensor_id = "5rTM-4sB-iVpGSRJcSJV_La Mata - Parque Natural_38.02519_-0.65845_0"
file_name = sensor_id.replace(" ", "_") + "_2024"
file_path = os.path.join(folder_path, file_name + ".json")

with open(file_path) as f:
    data = json.load(f)

In [ ]:
df_temperature = pd.DataFrame(data['series']['temperatura'][sensor_id], columns=['timestamp', 'temperature'])
df_humidity = pd.DataFrame(data['series']['humedad'][sensor_id], columns=['timestamp', 'humidity'])
df_pressure = pd.DataFrame(data['series']['presion'][sensor_id], columns=['timestamp', 'pressure'])

df_temperature.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_humidity.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_pressure.drop_duplicates(subset='timestamp', keep='first', inplace=True)

df_weather = pd.merge(df_temperature, df_humidity, how='inner', on='timestamp')
df_weather = pd.merge(df_weather, df_pressure, how='inner', on='timestamp')

df_weather.set_index('timestamp', inplace=True)

In [ ]:
del df_temperature, df_humidity, df_pressure
df_weather.to_csv(file_name + ".csv")

## Sonometer

Data about noise measured by a sonometer in decibels.

Data interval: 5 minutes.

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "noise": {
            "sensor-id": [
                [ "unix-timestamp", "value" ]
            ],
        },
        "humidity": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        },
        "temperature": {
            "sensor-name": [
                [ "unix-timestamp", "value" ]
            ],
        },
    }
}
```

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `noise`

`humidity` and `temperature` will be ignored, because are already present in the weather station data.

In [ ]:
folder_path = "json_data/sonometer"
file_name = "sonometros_dataset_l1_2024"
file_path = os.path.join(folder_path, file_name + ".json")

with open(file_path) as f:
    data = json.load(f)

In [ ]:
sensor_id = list(data['series']['noise'].keys())[0]

df_noise = pd.DataFrame(data['series']['noise'][sensor_id], columns=['timestamp', 'noise'])

df_noise.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_noise.set_index('timestamp', inplace=True)

In [ ]:
df_noise.to_csv(file_name + ".csv")

## Buoy

Data about water temperature measured in degrees Celsius at different depths (0.2 and 0.7 meters).

Data interval: 10 minutes.

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "temperature_a": {
            "sensor-1": [
                [ "unix-timestamp", "value" ]
            ],
            "sensor-2": [
                [ "unix-timestamp", "value" ]
            ],
            "..."
        },
        "temperature_b": {
            "..."
        }
    }
}
```

Here, data about water temperature is extracted from `temperature_a`, with 2 different sensors, based on the depths of 0.2 and 0.7 meters respectively.

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `temperature_02_meters`
- `temperature_07_meters`

In [ ]:
folder_path = "json_data"
file_name = "boya_dataset"
file_path = os.path.join(folder_path, file_name + ".json")

id_sensor_02_meters = "UVztVI0BaNT-uedBhHl1_Profundidad: -0.2m_38.03635_-0.68998_-2.2"
id_sensor_07_meters = "UVztVI0BaNT-uedBhHl1_Profundidad: -0.7m_38.03635_-0.68998_-2.7"

with open(file_path) as f:
    data = json.load(f)

In [ ]:
df_sensor_02 = pd.DataFrame(data['series']['temperature_a'][id_sensor_02_meters], columns=['timestamp', 'temperature_02'])
df_sensor_07 = pd.DataFrame(data['series']['temperature_a'][id_sensor_07_meters], columns=['timestamp', 'temperature_07'])

df_sensor_02.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_sensor_07.drop_duplicates(subset='timestamp', keep='first', inplace=True)

df_water_temperature = pd.merge(df_sensor_02, df_sensor_07, how='inner', on='timestamp')
df_water_temperature.set_index('timestamp', inplace=True)

### Data cleaning

Some temperature measures are invalid (< -10 degrees), so they will be removed.

In [ ]:
df_water_temperature = df_water_temperature[(df_water_temperature['temperature_02'] >= -10.0) & (df_water_temperature['temperature_07'] >= -10.0)]

In [ ]:
del df_sensor_02, df_sensor_07
df_water_temperature.to_csv(file_name + ".csv")

## Lagoon depth

Data about the depth of the lagoon measured in cm.

Data interval: 1 hour

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "profundidad": {
            "sensor-id": [
                [ "unix-timestamp", "value" ]
            ],
        }
    }
}
```

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `depth`

In [ ]:
folder_path = "json_data"
file_name = "profundidadlaguna_dataset"
file_path = os.path.join(folder_path, file_name + ".json")

sensor_id = "jRCJ548B1ljoqTiFLu0b_mqtt_consumerNivel laguna_38.02050_-0.66930_1"

with open(file_path) as f:
    data = json.load(f)

In [ ]:
df_lagoon_depth = pd.DataFrame(data['series']['profundidad'][sensor_id], columns=['timestamp', 'depth'])

df_lagoon_depth.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_lagoon_depth.set_index('timestamp', inplace=True)

### Data cleaning

Measures that differ more that a `threshold` from the previous one will be removed.

In [ ]:
threshold = 10

df_lagoon_depth['diff'] = df_lagoon_depth['depth'].diff()
df_lagoon_depth['diff'] = df_lagoon_depth['diff'].abs()
df_lagoon_depth = df_lagoon_depth[df_lagoon_depth['diff'] <= threshold]

df_lagoon_depth.drop(columns=['diff'], inplace=True)

In [ ]:
df_lagoon_depth.to_csv(file_name + ".csv")

## Visitors counter

Data about the numeber of visitors in the park.

Data interval:
- 1.5 minutes for sensors 1
- 5 minutes for sensors 2

### Input format

JSON file has the following structure:

```json
{
    "series": {
        "sensor-id": {
            "count-in": [
                [ "unix-timestamp", "value" ]
            ],
            "count-out": [
                [ "unix-timestamp", "value" ]
            ]
        }
    }
}
```

### Output format

The output will be a CSV file with the following columns:

- `timestamp`
- `count_in_sensor1`
- `count_out_sensor1`
- `count_in_sensor2`
- `count_out_sensor2`

In [ ]:
folder_path = "json_data"
file_name_sensor1 = "api_response1_entrance_outdoor"
file_name_sensor2 = "api_response2_cementery_outdoor"
file_path_sensor1 = os.path.join(folder_path, file_name_sensor1 + ".json")
file_path_sensor2 = os.path.join(folder_path, file_name_sensor2 + ".json")

with open(file_path_sensor1) as f:
    data_sensor1 = json.load(f)

with open(file_path_sensor2) as f:
    data_sensor2 = json.load(f)

In [ ]:
df_visitors_in_sensor1 = pd.DataFrame(data_sensor1['series']['hup4zYsBoTGoLusGWiIj_centro_38.02516_-0.65839_8']['count_in'], columns=['timestamp', 'count_in'])
df_visitors_out_sensor1 = pd.DataFrame(data_sensor1['series']['hup4zYsBoTGoLusGWiIj_centro_38.02516_-0.65839_8']['count_out'], columns=['timestamp', 'count_out'])

df_visitors_in_sensor1.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_visitors_out_sensor1.drop_duplicates(subset='timestamp', keep='first', inplace=True)

df_visitors_sensor1 = pd.merge(df_visitors_in_sensor1, df_visitors_out_sensor1, how='inner', on='timestamp')
df_visitors_sensor1.set_index('timestamp', inplace=True)

# df_visitors_sensor1["date"] = pd.to_datetime(df_visitors_sensor1.index, unit='ms')
# df_visitors_sensor1

In [ ]:
df_visitors_in_sensor2 = pd.DataFrame(data_sensor2['series']['5Ql2zYsBW9nuZhXkjyc1_centro_38.02516_-0.65839_8']['count_in'], columns=['timestamp', 'count_in'])
df_visitors_out_sensor2 = pd.DataFrame(data_sensor2['series']['5Ql2zYsBW9nuZhXkjyc1_centro_38.02516_-0.65839_8']['count_out'], columns=['timestamp', 'count_out'])

df_visitors_in_sensor2.drop_duplicates(subset='timestamp', keep='first', inplace=True)
df_visitors_out_sensor2.drop_duplicates(subset='timestamp', keep='first', inplace=True)

df_visitors_sensor2 = pd.merge(df_visitors_in_sensor2, df_visitors_out_sensor2, how='inner', on='timestamp')
df_visitors_sensor2.set_index('timestamp', inplace=True)

# df_visitors_sensor2["date"] = pd.to_datetime(df_visitors_sensor2.index, unit='ms')
# df_visitors_sensor2

### Considerations on merging sensors data

Both sensors have different data timestamp, so a pure merge join is not possible. The idea is to merge the timestamp with the nearest one.

In [ ]:
df_visitors_sensor1.sort_index(inplace=True)
df_visitors_sensor2.sort_index(inplace=True)

df_visitors = pd.merge_asof(df_visitors_sensor1, df_visitors_sensor2, on='timestamp', direction='nearest', suffixes=('_sensor1', '_sensor2'))
df_visitors

In [ ]:
df_visitors.to_csv("visitors.csv")